### INTEREST RATE TREE CALIBRATION (Black-Derman-Toy model)
Idea: we need a tree that correctly prices zero-coupon bonds (in
which case it correctly prices forwards and swaps) and options.

In order to calibrate the BDT Model we require:

- the observed bond prices at time 0 
- the volatility of bond yields

Let us label each node of the tree (n, j)
- $n = 0,1,2,...,N$ (time)
- $j = 0,1,2,...,n$ (number of heads)

Then define $AD(n,j)$ to be the time zero price of Arrow-Debreu security at node $(n, j)$, i.e. AD pays $\$1$ if the process reaches node $(n,j)$

In [1]:
import math
import pandas as pd
import numpy as np
import sympy as sp

In [2]:
data = pd.DataFrame(index = range(1,5), 
                    columns = ['Yield', 'Bond Price', 'Yield Vol'],
                    data = [[0.1,0.9091, np.NaN], 
                            [0.11, 0.8116, 0.10], 
                            [0.12, 0.7118, 0.15], 
                            [0.125, 0.6243, .14] ])
data

,Yield,Bond Price,Yield Vol
1,0.100,0.9091,NaN
2,0.110,0.8116,0.10
3,0.120,0.7118,0.15
4,0.125,0.6243,0.14


#### Model parameters

In [3]:
N = data.shape[0]
p_tilda = 1/2

#### Initialize Probability, Rates and Arrow-Debreu matrices
$\tilde{\mathbb P}$ is assumed to be $\frac{1}{2}$ in BDT model

In [4]:
"probability matrix"
P = np.full((N+1,N+1), p_tilda) # allows for probabilities different from 1/2

"rates matrix"
R = np.zeros((N, N))
R[0,0] = data['Yield'][1]

AD = np.zeros((N, N, N, N)) 
AD[0,0,0,0] = 1

TIME_ZERO_BOND_PRICES = []

#### Risk Neutral Pricing Formula (one-step back prices)
Returns the price of Arrow-Debreu security at node $(n-1,i)$ that pays $1\$$ at the node $(n,j)$

In [5]:
def martingale_pricing(i,n,j):
    if i == j - 1:
        ad = P[n-1,j-1]/(1+R[n-1,j-1])
    elif i == j:
        ad = (1-P[n-1,j])/(1+R[n-1,j])
    else:
        ad = 0
    AD[n-1,i,n,j] = ad
#     print('A({},{},{},{})'.format(n-1,i,n,j), ad)

#### Jamshidian Forward Induction

returns price of Arrow-Debreu security at node (n,i) that pays $1\$$ at node $(m,j)$ (i.e $j$ heads in first $m$ coin tosses)

In [6]:
def jfi(n,i,m,j):
    if j == 0:
        ad = AD[n,i,m-1,j] * (1-P[m-1][j])/(1+R[m-1][j])
    elif 0 < j & j < m:
        ad = AD[n,i,m-1,j] * (1-P[m-1][j])/(1+R[m-1][j]) + AD[n,i,m-1,j-1] * (P[m-1][j-1])/(1+R[m-1][j-1])
    elif j == m:
        ad = AD[n,i,m-1,j-1] * (P[m-1][j-1])/(1+R[m-1][j-1])
    AD[n,i,m,j] = ad

The calculation of the rate $R(n,j+1)$ is based on the assumption that rate volatilities satisfy the following condition:

$$\sigma_r(n) = \frac{1}{2}\log\left( \frac{R(n,j+1)}{R(n,j)} \right) \ \ \ or\ \ \ R(n, j+1) = R(n,j) e^{2\sigma_r(n)} = R(n,j) \sigma(n)$$


In [7]:
def rate_vol(x):
    return (1/2) * math.log(x)

We also assume that the given yield volatilities satisfy:

$$ \sigma_y(n) = \frac{1}{2}\log\left( \frac{Y(1,1;n)}{Y(1,0;n)} \right) \ \ \ or\ \ \ Y(1,1;n) = Y(1,0;n) e^{2\sigma_y(n)} \ \ \ for \ n = 2,3,...$$

We will also create a matrix to store time 1 yields for different maturities.


In [8]:
yield_tree = np.zeros((2,N)) 

We now have everything we need to calibrate our model

In [9]:
for n in range(1,N): 
    
    '''
    We need two solve a system of two equations in two unknowns:
        - interest rate at time i with j = 0 
        - interest rate volatility at time i
        
    We can express the unknown variables in two ways:
        (1) in terms of time 0 bond yields and time 0 prices of AD securities 
        (2) in terms of time 1 yields of bonds maturing at time n+1 
            and time 1 prices of AD securities
    '''

    r = sp.Symbol('r')
    sigma = sp.Symbol('sigma') 
    
    ##############
    # EQUATION 1 #   
    ##############
    
    "fill a level of time 0 AD tree using JFI"
    for j in range(0,n+1):
        jfi(0,0,n,j)
        
    "we calibrate to the observed time 0 bond price"
    bond_price = pow(1+data['Yield'][n+1], -(n+1))
    TIME_ZERO_BOND_PRICES.append(bond_price)
    
    if n == 1:
        "at time 1 rates are the same as yields of bonds at time 1 with maturity 2"
        rate_volatility = math.exp(2*data['Yield Vol'][n+1])
        B = sum( [AD[0,0,n,j]/(1 + r * pow(rate_volatility, j)) for j in range(0,n+1)])
        rate = np.array([sp.re(x) for x in sp.solvers.solve(bond_price - B, r)])
        rate = rate[rate>0][0] # set the interest rate to the positive root
    else:
        "after time one we have to construct a system to solve for numerically"
        "we use time 0 prices of AD securities summed over possible coin toss outcomes"
        B = sum([AD[0,0,n,j]/(1 + r * pow(sigma, j)) for j in range(0,n+1)])
        
    ##############
    # EQUATION 2 #   
    ##############
    
    if n > 1: 
        "calculate time 1 yields of bonds"
        Y = [] 
        for i in range(0,2): # yields at (1,0;n+1) and (1,1;n+1) 
            p = 0 # bond prices
            for j in range(0,n+1): # payoffs at terminal nodes: (n,0), (n,1), ... , (n,n)
                "calculate one step AD prices using risk neutral pricing formula"
                martingale_pricing(i,n,j)
                if n > 2:
                    jfi(1,i,n,j)
                p += ( AD[1,i,n,j] )/(1 + r * pow(sigma, j)) #P[1,i,n+1]
            Y.append(pow(p,-(1/n)) - 1)

    ####################
    # SOLVE THE SYSTEM #   
    ####################
    
        solution = sp.nsolve(
            [Y[0] * math.exp(2*data['Yield Vol'][n+1]) - Y[1], bond_price - B], 
            [r,sigma], [0.08, 0.01])
        rate = solution[0]
        rate_volatility = solution[1]
        
    "substitute the values found to calculate actual time 1 yields"
    if n > 1:
        for i in range(0,2):
            p = 0 # bond prices
            for j in range(0,n+1): # payoffs at terminal nodes of time n: (n,0), (n,1), ... , (n,n)
                p += ( AD[1,i,n,j] )/(1 + rate * pow(rate_volatility, j)) #P[1,i,n+1]
            yield_tree[i,n] = pow(p,-(1/n)) - 1
  
    print('Time {} rate volatility is {}'.format(n, rate_vol(rate_volatility)))
    
    R[n][0] = rate # save the rate to the tree
    'compute rest of the interests rates applying BDT condition for volatility'
    for j in range(1,n+1):
        R[n][j] = R[n][0] * pow(rate_volatility, j)
print(R) 

Time 1 rate volatility is 0.1
Time 2 rate volatility is 0.19478447273332103
Time 3 rate volatility is 0.12227923998917059
[[0.1        0.         0.         0.        ]
 [0.10823708 0.13220106 0.         0.        ]
 [0.09254136 0.1366229  0.20170244 0.        ]
 [0.09616446 0.12280753 0.15683226 0.20028379]]


## PART B
To verify that the 1-year yield volatility of the 4-year zero coupon bond in the calibrated model is $0.14$ we can check if the volatility condition of BDT model is satifsied. That is, we should have:

$$ \sigma_y(4) = \frac{1}{2}\log\left( \frac{Y(1,1;4)}{Y(1,0;4)} \right)$$

In [10]:
data.loc[4,'Yield Vol']

0.14

In [11]:
math.log(yield_tree[1,3]/yield_tree[0,3])/2

0.13999999999999982

## PART C

Let us verify that the price at time zero of the 12% interest rate cap on $100
notional principal 4-year loan is 3.909.

Recall that risk-neutral price of an m-period interaste cap is given by:

$$ \operatorname{Cap}_m = \tilde{\mathbb E}\left[ \sum_{n=1}^m D_n(R_{n-1} - K)^+ \right] =
\tilde{\mathbb E}\left[ D_1(R_{0}-K)^+ + D_2(R_1-K)^+ D_3(R_2-K)^+ + D_4(R_3-K)^+  \right] \\
= \tilde{\mathbb E}\left[ D_1(R_{0}-K)^+ \right] + \tilde{\mathbb E}\left[ D_2(R_1-K)^+ \right] + \tilde{\mathbb E}\left[ D_3(R_2-K)^+ \right] + \tilde{\mathbb E}\left[ D_4(R_3-K)^+ \right] \\
= \tilde{\mathbb E}\left[ \frac{(R_{0}-K)^+}{1+R_0} \right] + \tilde{\mathbb E}\left[ \frac{(R_{1}-K)^+}{(1+R_0)(1+R_1)} \right] + \tilde{\mathbb E}\left[ \frac{(R_{2}-K)^+}{(1+R_0)(1+R_1)(1+R_2)} \right] + \tilde{\mathbb E}\left[ \frac{(R_3-K)^+}{(1+R_0)(1+R_1)(1+R_2)(1+R_3)} \right] \\
$$

In [12]:
"gets paths from root to a terminal node for a given depth and a starting point"
def getPaths(depth, targetVal, curVal = 0):
#     print(depth, targetVal, curVal)
    if 0 == depth:
        return [[curVal]] if targetVal == curVal else []

    leftPaths = getPaths(depth - 1, targetVal, curVal)
#     print('pass', depth, targetVal, curVal, leftPaths )
    rightPaths = getPaths(depth - 1, targetVal, curVal + 1)
    return [[curVal] + path for path in leftPaths + rightPaths]

In [13]:
caplets = []
K = 0.12 # target rate

for i in range(0,N+1): 
    
    """"
    We shall consider all the possible paths leading to a terminal node,
    since the interest rates are path dependent (and so could be the probabilities)
    """
    
    for leaf in range(0,2**i):
        paths = getPaths(i, leaf)
        if paths == []:
            continue
        
        E = 0 # time 0 expected price of a caplet paying off at time i 
        for path in paths: 
            D = 1
            probability = 1
            for j in range(0, i):
                D *= (1 + R[j, path[j]]) # aggregate discounting factors
                probability *= P[j, path[j]] # compute probability of the path
            payoff = max(0, R[i-1,path[i-1]] - K) # caplet's payoff for this path
            E += pow(D, -1) * payoff * probability # increment expeceted value of the path         
        caplets.append(E)
sum(caplets)

0.039091600135354974

## PART D
Forward rate $F_{n,m} = \frac{B_{n,m}}{B_{n,m+1}} -1 $
is the rate, locked in at time n, at which we can invest $1 between time m
and m + 1 for one period.

We can to verify that $F_{0,3} = \frac{B_{0,3}}{B_{0,4}} -1 = 0.140134$

In [14]:
TIME_ZERO_BOND_PRICES[1]/TIME_ZERO_BOND_PRICES[2] - 1

0.14013432761322964